In [1]:
import gym
import matplotlib.pyplot as plt
import random
import numpy as np

In [ ]:
env = gym.make('Blackjack-v0')
env.reset()

In [ ]:
a = [0,1,2,3]
b = a[:2]
print(a)
print(b)
print(observation)
print(observation[3])

In [ ]:
total_payout = 0 # to store total payout over 'num_rounds'
average_payouts = []
agent = Agent(env=env, epsilon=1.0, gamma=0.9, num_episodes_to_train=500, timestep=0)

num_rounds = 800 # Payout calculated over num_rounds
num_samples = 1 # num_rounds simulated over num_samples


observation = env.reset()
for sample in range(num_samples):
    round = 1
    epsilon_values = []
    # Take action based on Q-table of the agent and learn based on that until 'num_episodes_to_train' = 0
    while round <= num_rounds:
        epsilon_values.append(agent.epsilon)
        action = agent.choose_action(observation)
        next_observation, payout, is_done, _ = env.step(action)
        agent.learn(observation, action, payout, next_observation)
        total_payout += payout
        observation = next_observation
        if is_done:
            agent.timestep += 1
            observation = env.reset() # Environment deals new cards to player and dealer
            round += 1
            average_payouts.append(total_payout/(sample*num_rounds + round))

# Plot epsilon over rounds to show rate of its decrease
plt.figure(2)
plt.xlabel("num_rounds")
plt.ylabel("epsilon")
plt.plot(epsilon_values)
plt.show()
print ("Average payout after {} rounds is {}".format(num_rounds, total_payout/(num_samples)))

In [ ]:
learnedStrategy = open('AgentStrategy.txt','w')

In [ ]:
num_rounds = 1000 # Payout calculated over num_rounds
num_samples = 100 # num_rounds simulated over num_samples

payouts = []

observation = env.reset()
round = 1
total_payout = 0 # to store total payout over 'num_rounds'
# Take action based on Q-table of the agent and learn based on that until 'num_episodes_to_train' = 0
while round <= num_rounds * num_samples:
    action = agent.choose_action(observation)
    next_observation, payout, is_done, _ = env.step(action)
    agent.learn(observation, action, payout, next_observation)
    payouts.append(payout)
    observation = next_observation
    if is_done:
        observation = env.reset() # Environment deals new cards to player and dealer
        round += 1

num_observations = 0        
list_players_hand = range(1, 22)
list_dealers_upcard = range(1, 11)

def readable_action(observation, agent):
    """ 
    Pass observation to agent and get human readable action
    H is hit, S is stick and '-' means the state is unseen and a random action is taken
    """
    if observation not in agent.Q:
        action = "-"
    else:
        strategy = agent.choose_action(observation)
        if strategy == 2:
            action = "D"
        elif strategy == 1:
            action = "H" 
        else:
            action = "S"    
    return action

# Print headers to give more information about output
learnedStrategy.write("{:^10} | {:^50} | {:^50}".format(
                      "Player's","Dealer's upcard when ace is not usable",
                      "Dealer's upcard when ace is usable"))
learnedStrategy.write("{0:^10} | {1} | {1}".format("Hand", [str(upcard) if not upcard==10 else 'A' 
                                                        for upcard in list_dealers_upcard]))
print (''.join(['-' for _ in range(116)]))
for players_hand in list_players_hand:
    actions_usable1, actions_usable2 = [], []
    actions_not_usable1, actions_not_usable2 = [], []
    for dealers_upcard in list_dealers_upcard:
        observation = (players_hand, dealers_upcard, False, True)
        actions_not_usable1.append(readable_action(observation, agent))
        observation = (players_hand, dealers_upcard, False, False)
        actions_not_usable2.append(readable_action(observation, agent))
        
        observation = (players_hand, dealers_upcard, True, True)
        actions_usable1.append(readable_action(observation, agent))
        observation = (players_hand, dealers_upcard, True, False)
        actions_usable2.append(readable_action(observation, agent))
    
    learnedStrategy.write("{:>10} | {} | {}".format(
                          players_hand, actions_not_usable1, actions_usable1))
    learnedStrategy.write("{:>10} | {} | {}".format(
                          players_hand, actions_not_usable2, actions_usable2))

    
learnedStrategy.write("Average payout after {} rounds is {}".format(
                       num_rounds, sum(payouts)/num_samples))

learnedStrategy.close()

In [ ]:
env.action_space.n

In [ ]:
agent.valid_actions

In [ ]:
agent = Agent(env=env, epsilon=1.0, gamma=0.1, num_episodes_to_train=15000, timestep=0)

num_rounds = 1000 # Payout calculated over num_rounds
num_samples = 1000 # num_rounds simulated over num_samples

average_payouts = []

observation = env.reset()
for sample in range(num_samples):
    round = 1
    total_payout = 0 # to store total payout over 'num_rounds'
    # Take action based on Q-table of the agent and learn based on that until 'num_episodes_to_train' = 0
    while round <= num_rounds:
        action = agent.choose_action(observation)
        next_observation, payout, is_done, _ = env.step(action)
        agent.learn(observation, action, payout, next_observation)
        total_payout += payout
        observation = next_observation
        if is_done:
            agent.timestep += 1
            observation = env.reset() # Environment deals new cards to player and dealer
            round += 1
    average_payouts.append(total_payout)

# Plot payout per 1000 episodes for each value of 'sample'
plt.plot(average_payouts)           
plt.xlabel('num_samples')
plt.ylabel('payout after 1000 rounds')
plt.show()      
    
print ("Average payout after {} rounds is {}".format(num_rounds, sum(average_payouts)/(num_samples)))

In [ ]:
import gym
from gym import spaces
from gym.utils import seeding

In [ ]:
def cmp(a, b):
    return float(a > b) - float(a < b)

# 1 = Ace, 2-10 = Number cards, Jack/Queen/King = 10
deck = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
deckVal = dict()
for card in deck:
    if card == 11 or card == 12 or card == 13:
        deckVal[card] = 10
    else:
        deckVal[card] = card

def draw_card(np_random):
    return int(np_random.choice(deck))


def draw_hand(np_random):
    return [draw_card(np_random), draw_card(np_random)]

def trueHandSum(hand):
    trueSum = 0
    for card in hand:
        trueSum += deckVal[card]
    return trueSum

def isSplited(hand):
    return type(hand[0]) == list

def split(hand):
    if isSplited(hand):
        hand.append(hand[0][0])
    else:
        hand.append([hand[0]])
    
    return hand
    
def usable_ace(hand):  # Does this hand have a usable ace?
    return 1 in hand and trueHandSum(hand) + 10 <= 21


def sum_hand(hand):  # Return current hand total
    if usable_ace(hand):
        return trueHandSum(hand) + 10
    return trueHandSum(hand)


def is_bust(hand):  # Is this hand a bust?
    return sum_hand(hand) > 21


def score(hand):  # What is the score of this hand (0 if bust)
    return 0 if is_bust(hand) else sum_hand(hand)


def is_natural(hand):  # Is this hand a natural blackjack?
    return sorted(hand) == [1, 10]


def is_firstRound(hand):
    return len(hand) == 2

def isPair(hand):
    return hand[0] == hand[1]


class BlackjackEnv(gym.Env):
    """Simple blackjack environment

    Blackjack is a card game where the goal is to obtain cards that sum to as
    near as possible to 21 without going over.  They're playing against a fixed
    dealer.
    Face cards (Jack, Queen, King) have point value 10.
    Aces can either count as 11 or 1, and it's called 'usable' at 11.
    This game is placed with an infinite deck (or with replacement).
    The game starts with each (player and dealer) having one face up and one
    face down card.

    The player can request additional cards (hit=1) until they decide to stop
    (stick=0) or exceed 21 (bust).

    After the player sticks, the dealer reveals their facedown card, and draws
    until their sum is 17 or greater.  If the dealer goes bust the player wins.

    If neither player nor dealer busts, the outcome (win, lose, draw) is
    decided by whose sum is closer to 21.  The reward for winning is +1,
    drawing is 0, and losing is -1.

    The observation of a 3-tuple of: the players current sum,
    the dealer's one showing card (1-10 where 1 is ace),
    and whether or not the player holds a usable ace (0 or 1).

    This environment corresponds to the version of the blackjack problem
    described in Example 5.1 in Reinforcement Learning: An Introduction
    by Sutton and Barto.
    http://incompleteideas.net/book/the-book-2nd.html
    """
    def __init__(self, natural=False):
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Tuple((
            spaces.Discrete(32),
            spaces.Discrete(11),
            spaces.Discrete(2),
            spaces.Discrete(2),
            spaces.Discrete(2)))
        self.seed()

        # Flag to payout 1.5 on a "natural" blackjack win, like casino rules
        # Ref: http://www.bicyclecards.com/how-to-play/blackjack/
        self.natural = natural
        # Start the first game
        self.reset()

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def step(self, action):
        assert self.action_space.contains(action)
        if isSplited(self.player)
        # split
        if action == 3:
            self.player = split(self.player)
        # double
        elif action == 2:
            self.player.append(draw_card(self.np_random))
            if is_bust(self.player):
                done = True
                reward = -2
            else:
                done = True
                while sum_hand(self.dealer) < 17:
                    self.dealer.append(draw_card(self.np_random))
                reward = 2*cmp(score(self.player), score(self.dealer))
        # hit: add a card to players hand and return
        elif action == 1:
            self.player.append(draw_card(self.np_random))
            if is_bust(self.player):
                done = True
                reward = -1
            else:
                done = False
                reward = 0
        # stick: play out the dealers hand, and score
        else:
            done = True
            while sum_hand(self.dealer) < 17:
                self.dealer.append(draw_card(self.np_random))
            reward = cmp(score(self.player), score(self.dealer))
            if is_natural(self.player) and reward == 1:
                reward = 1.5
        return self._get_obs(), reward, done, {}

    def _get_obs(self):
        return (sum_hand(self.player), self.dealer[0], usable_ace(self.player), is_firstRound(self.player), isPair(self.player))

    def reset(self):
        self.dealer = draw_hand(self.np_random)
        self.player = draw_hand(self.np_random)
        return self._get_obs()

In [8]:
v = np.array([[1,2],[3,2],[4,2]])
print(v)
print(np.shape(v))

[[1 2]
 [3 2]
 [4 2]]
(3, 2)


In [ ]:
class Agent():
    def __init__(self, env, epsilon=1.0, gamma=0.99, num_episodes_to_train=30000, timestep=0):
        self.env = env

        # Looks like n is number of valid actions from the souce code
        self.valid_actions = list(range(self.env.action_space.n))

        # Set parameters of the learning agent
        self.Q = dict()          # Q-table which will be a dictionary of tuples
        self.epsilon = epsilon   # Random exploration factor
        #self.alpha = alpha       # Learning factor
        self.gamma = gamma       # Discount factor- closer to 1 learns well into distant future
        self.visits = dict()
        
        # epsilon will reduce linearly until it reaches 0 based on num_episodes_to_train
        # epsilon drops to 90% of its inital value in the first 30% of num_episodes_to_train
        # epsilon then drops to 10% of its initial value in the next 40% of num_episodes_to_train
        # epsilon finally becomes 0 in the final 30% of num_episodes_to_train
        self.num_episodes_to_train = num_episodes_to_train # Change epsilon each episode based on this
        #self.small_decrement = (0.1 * epsilon) / (0.3 * num_episodes_to_train) # reduces epsilon slowly
        #self.big_decrement = (0.8 * epsilon) / (0.4 * num_episodes_to_train) # reduces epilon faster

        #self.num_episodes_to_train_left = num_episodes_to_train
        self.timestep = timestep
        
    def updateEpsilon(self):
        epsilonStart = 1
        epsilonEnd = 0
        scalingFactor = 100
        self.epsilon = epsilonStart + (epsilonEnd-epsilonStart)*np.exp(
                       (self.timestep-self.num_episodes_to_train)/scalingFactor)

    def update_parameters(self):
        """
        Update epsilon and alpha after each action
        Set them to 0 if not learning
        """
        '''
        if self.num_episodes_to_train_left > 0.7 * self.num_episodes_to_train:
            self.epsilon -= self.small_decrement
        elif self.num_episodes_to_train_left > 0.3 * self.num_episodes_to_train:
            self.epsilon -= self.big_decrement
        elif self.num_episodes_to_train_left > 0:
            self.epsilon -= self.small_decrement
        else:
            self.epsilon = 0.0
            self.alpha = 0.0
        '''
        #self.timestep += 1
        if self.timestep <= self.num_episodes_to_train:
            self.updateEpsilon()
        else:
            self.epsilon = 0

    def create_Q_if_new_observation(self, observation):
        """
        Set intial Q values to 0.0 if observation not already in Q table
        """
        # If not first round, double and split is not allowed
        if observation[3] == False:
            self.valid_actions = self.valid_actions[:2]
        else:
            # If the player has a pair
            if observation[4] == True:
                self.valid_actions = list(range(self.env.action_space.n))
            else:
                self.valid_actions = self.valid_actions[:3]

        if observation not in self.Q:
            self.Q[observation] = dict((action, 0.0) for action in self.valid_actions)
            self.visits[observation] = dict((action, 1) for action in self.valid_actions)

    def get_maxQ(self, observation):
        """
        Called when the agent is asked to find the maximum Q-value of
        all actions based on the 'observation' the environment is in.
        """
        self.create_Q_if_new_observation(observation)
        return max(self.Q[observation].values())

    def choose_action(self, observation):
        """
        Choose which action to take, based on the observation.
        If observation is seen for the first time, initialize its Q values to 0.0
        """
        
        self.create_Q_if_new_observation(observation)

        # uniformly distributed random number > epsilon happens with probability 1-epsilon
        if random.random() > self.epsilon:
            maxQ = self.get_maxQ(observation)

            # multiple actions could have maxQ- pick one at random in that case
            # this is also the case when the Q value for this observation were just set to 0.0
            action = random.choice([k for k in self.Q[observation].keys()
                                    if self.Q[observation][k] == maxQ])
        else:
            action = random.choice(self.valid_actions)
        
        self.visits[observation][action] += 1
        self.update_parameters()

        return action


    def learn(self, observation, action, reward, next_observation):
        """
        Called after the agent completes an action and receives an award.
        This function does not consider future rewards
        when conducting learning.
        """
        
        # Q = Q*(1-alpha) + alpha(reward + discount * utility of next observation)
        # Q = Q - Q * alpha + alpha(reward + discount * self.get_maxQ(next_observation))
        # Q = Q - alpha (-Q + reward + discount * self.get_maxQ(next_observation))
        alpha = 1/(1+self.visits[observation][action])
        self.Q[observation][action] = (1-alpha)*self.Q[observation][action] + (
                                      (alpha)*(reward+(self.gamma*self.get_maxQ(next_observation))))